In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
!pip install ar_corrector
from ar_corrector.corrector import Corrector
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
import numpy as np

     |████████████████████████████████| 53.4 MB 18.0 MB/s 


In [3]:
BookName=[]
authers=[]
deps=[]
page_no=[]
publisher=[]
lang=[]
Links=[]
description=[]

In [4]:
def deleteDublicatedNames(string):
        splitedName=[]
        name=string
        if 'كتب' in string.split():
            splitedName=string.split('كتب')
        elif 'الكاتب' in string.split():
            splitedName=string.split('الكاتب')
            #print(splitedName)
        splitedName = [i for i in splitedName if i]
            #print(splitedName)
        if len(splitedName)>0:
            name=splitedName[0]
        return " ".join(name.replace('كتب','').replace('الكاتب','').split())

In [5]:
def AppendDataRecords(name,text):
    authers.append(deleteDublicatedNames(" ".join((text[0].text[12:]).replace('\n'," ").split())))
    lang.append(" ".join((text[2].text[11:]).split()))
    NoOfPages=(text[3].text[14:])
    numeric_filter=filter(str.isdigit,NoOfPages)
    numeric_string="".join(numeric_filter)
    if(numeric_string==''):
        numeric_string='10'
    page_no.append(int(numeric_string))
    publisher.append(" ".join((text[4].text[11:]).replace('دار','').replace('مؤسسة','').replace('\n'," ").split())) 
    BookName.append(" ".join(((name.replace('كتاب','')).replace('PDF','')).replace('\n'," ").split()))

In [6]:
def no_OfRecords(ReadLink,name,text,desc):
    dep=((text[1].text[13:]).replace( 'تحميل','').replace('كتب','').replace('\n\n','\n'))
    ListOfDep=dep.split('\n')
    for x in ListOfDep:
        if x!='' :
            deps.append(" ".join(x.split()))
            Links.append(ReadLink)
            description.append(desc.replace('PDF',''))
            AppendDataRecords(name,text)

In [7]:
url='https://www.arab-books.com'
corr = Corrector()
for i in range(1,21):
    #print('page'+str(i))
    req=requests.get(url+'/page'+'/'+str(i)+'/')
    content=req.text
    soup=BeautifulSoup(content)
    ul=str(soup.findAll('ul', attrs={'class':'posts-items'}))
    ul=BeautifulSoup(ul)
    for a in ul.find_all('a',href=True, attrs={'class':'post-thumb'}):
        name=a.get('aria-label')
        link=a.get('href')
        req=requests.get(link)
        content=req.text
        content=BeautifulSoup(content)
        data=str(content.find_all('div', attrs={'class':'book-info'}))
        data=BeautifulSoup(data)
        parent=data.find('ul')
        text=parent.find_all("li")
        div=str(content.find_all('div',attrs={'class':'read-link-bottom'}))
        div=BeautifulSoup(div)
        link=div.find('a',href=True)
        if (not link):
            ReadLink='Not Avialable'
        else : ReadLink=link.get('href')
        div=str(content.find_all('div',attrs={'class':'entry-content entry clearfix'}))
        div=BeautifulSoup(div)
        rdiv=div.find_all('p')
        if len(div.find_all('p'))>=3 :
            desc=corr.contextual_correct(rdiv[2].text.replace('PDF',''))
        else:
            desc=" "
        no_OfRecords(ReadLink,name,text,desc)

In [8]:
dic={'اسم الكتاب': BookName ,'الكاتب': authers , 'القسم': deps , 'عدد صفحات الكتاب': page_no , 'اللغه' : lang , 'دار النشر':publisher ,'رابط قراءه الكتاب' : Links ,'ملخص الكتاب' : description }
df = pd.DataFrame(dic)
df.to_csv('DataSet.csv',index=False,header=True,encoding='utf-8-sig')

In [9]:
df=pd.read_csv("DataSet.csv",encoding= 'utf-8-sig')

In [10]:
columns=['اسم الكتاب','الكاتب','القسم','ملخص الكتاب']
df=df[columns]
df

,اسم الكتاب,الكاتب,القسم,ملخص الكتاب
0,لغز القبر الملكي,محمود سالم,روايات بوليسية,محمود سالم
1,لغز الكوخ المحترق,محمود سالم,روايات بوليسية,محمود سالم
2,لغز اللص الشبح,محمود سالم,روايات بوليسية,محمود سالم
3,لغز المدينة العائمة,محمود سالم,روايات بوليسية,محمود سالم
4,لغز المنزل رقم 98,محمود سالم,روايات بوليسية,محمود سالم
...,...,...,...,...
676,مجلة ميكي العدد 536,والت ديزني,الأطفال,ولد والتر إلياس ديزني المؤلف البارع لسلسلة مجل...
677,مجلة ميكي العدد 530,والت ديزني,الأطفال,ولد والتر إلياس ديزني المؤلف البارع لسلسلة مجل...
678,مجلة ميكي العدد 527,والت ديزني,الأطفال,ولد والتر إلياس ديزني المؤلف البارع لسلسلة مجل...
679,مجلة ميكي العدد 526,والت ديزني,الأطفال,ولد والتر إلياس ديزني المؤلف البارع لسلسلة مجل...


In [11]:
le = preprocessing.LabelEncoder()
x=[[],[]]
y=[]
x[0]=le.fit_transform(df.iloc[:,0])
x[1]=le.fit_transform(df.iloc[:,1])
y=le.fit_transform(df.iloc[:,2])
x=np.transpose(x)

In [12]:
columns=['اسم الكتاب','الكاتب','القسم']
df=df[columns]
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.3)
gnb = GaussianNB()
gnb.fit(x_train, y_train)
y_pred = gnb.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5609756097560976
